In [ ]:

import sys
import os
import logging
import yaml
import torch
from google.colab import drive
import pandas as pd
import numpy as np
import random

drive.mount('/content/drive')

PROJECT_DIR = "/content/drive/MyDrive/BiologicalData/progetto/biological_data_pfp/biological_data_pfp"
SRC_DIR = f"{PROJECT_DIR}/src"
CONFIG_FILE = f"{PROJECT_DIR}/notebooks/flavio/config.yaml"
OUTPUT_FILE = f"{PROJECT_DIR}/results/submission.tsv"

sys.path.append(SRC_DIR)

# Choose device
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Device: {device}")

# Load the configuration data from the YAML file
with open(CONFIG_FILE, 'r') as f:
    config_data = yaml.safe_load(f)


np.random.seed(42)
random.seed(42)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Device: cpu


In [ ]:
blast_folder = f"{PROJECT_DIR}/train/blast_results_train"
df_gt = pd.read_csv(f"{PROJECT_DIR}/train/tsvs/train_set.tsv", sep='\t')

In [ ]:

def create_count_df(df):
    # this function is to transform a table with blast results in a table of counts
    df = df.explode('GO_term')
    df = df.groupby([0, 'GO_term']).size().unstack(fill_value=0)
    df.reset_index(inplace=True)
    return df

def blast_dfs_aspect(df_blast, df_gt, aspect, thresh=0.001):
    df_blast = df_blast.drop([2, 4], axis=1)                              # drop useless columns
    df_blast = df_blast.loc[df_blast[3] <= thresh]                        # apply evalue threshold
    df_blast = df_blast.drop([3], axis=1)                                 # drop useless column
    df_blast = df_blast.drop_duplicates(subset=[0, 1])                    # eliminate duplicates (won't count the same coupling more than once)
    df_blast = df_blast[df_blast[0] != df_blast[1]]                       # eliminate self alignment (protein with itself)
    df_blast = df_blast.merge(df_gt, left_on=1, right_on='Protein_ID', how='left')      # merge with ground truth to get GO annotations
    df_blast = df_blast.groupby([0, 'aspect'], as_index=False).agg({'GO_term': lambda x: list(x)})      # groupby aspect
    df_blast = df_blast[df_blast['aspect'] == aspect]                     # select aspect of interest
    df_blast = df_blast.drop(['aspect'], axis=1)                          # drop aspect column
    df_blast = create_count_df(df_blast)                                  # create counts table with counts fo each GO

    return df_blast


In [ ]:

# the following procedure has to be repeated for each aspect
chunk = []
aspect = 'biological_process'

# go through the blast files and get table of counts for each. store each resulting pd in an array
for file in os.listdir(blast_folder):
    filename = os.fsdecode(file)
    print(filename)
    if filename.endswith(".tsv"):
        df_blast = pd.read_csv(f"{blast_folder}/{filename}", header=None, sep='\t')
        df_blast = blast_dfs_aspect(df_blast, df_gt, aspect, thresh=0.001)

        chunk.append(df_blast)
    else:
        continue
# uncomment when needed. concat chunks stored in array
#df_blast_bp_all = pd.concat(chunk, ignore_index=True)
#df_blast_mf_all = pd.concat(chunk, ignore_index=True)
#df_blast_cc_all = pd.concat(chunk, ignore_index=True)


train.part_004.tsv
train.part_005.tsv
train.part_006.tsv
train.part_007.tsv
train.part_008.tsv
train.part_009.tsv
train.part_010.tsv
train.part_011.tsv
train.part_012.tsv
train.part_013.tsv
train.part_014.tsv
train.part_015.tsv
train.part_016.tsv
train.part_017.tsv
train.part_018.tsv
train.part_019.tsv
train.part_020.tsv
train.part_021.tsv
train.part_022.tsv
train.part_023.tsv
train.part_024.tsv
train.part_025.tsv
train.part_001.tsv
train.part_002.tsv
train.part_003.tsv


In [ ]:
# save to file
combined_df.to_csv('combined_df.tsv', sep='\t', index=False, header = True)

!mkdir blast_dfs
!mv 'combined_df.tsv' blast_dfs/
!cp -r "blast_dfs" "$PROJECT_DIR/train/"

mkdir: cannot create directory ‘blast_dfs’: File exists
